In [20]:
import json
import os
import pandas as pd
import numpy as np

trialLength = 3300
baselineLength = 1000

In [21]:
def calculate_column_wise_average(df):
    columns_to_average = df.columns[5:3305].values
    return df[columns_to_average].mean(axis=0)

In [26]:
all_participants_data = pd.DataFrame()

def process_participant(participantID):
    global all_participants_data 
    path = "/Users/dhewitt/Data/pps/"
    
    PKL_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_dataframelist.pkl")
    CSV_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_Extractednew.csv")
    #OUT_FILE_NAME = os.path.join(path + participantID + "/" + participantID + "_HandDataProcessed.csv")
    AV_OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_HandDataProcessedAv_2911.csv")
    
    data2load = pd.read_csv(CSV_FILE_NAME) 

    dataframe_list = pd.read_pickle(PKL_FILE_NAME)

    #Interpolating the df due to changes in sampling rate for collecting data between trials.
    #Normalising the timestamp from the first timepoint.
    #Just getting the 'in trial' bit from trial start/cue appearing to the pain stimulus onset.
    
    def extract_and_interpolate_hand_position(df, trial_number):
        left_hand_position = df['left_hand_position'].apply(pd.Series)  # Expand tuple into separate columns
        interpolated_hand_position = pd.DataFrame({
            'hand_position_x': np.interp(np.arange(baselineLength + trialLength),
                                          df['timestamp'] - df['timestamp'].iloc[0],
                                          left_hand_position[0]),
            'hand_position_y': np.interp(np.arange(baselineLength + trialLength),
                                          df['timestamp'] - df['timestamp'].iloc[0],
                                          left_hand_position[1]),
            'hand_position_z': np.interp(np.arange(baselineLength + trialLength),
                                          df['timestamp'] - df['timestamp'].iloc[0],
                                          left_hand_position[2]),
            'trial': trial_number
        })
        return interpolated_hand_position

    # Apply the function to each DataFrame in the list
    hand_position_data = [extract_and_interpolate_hand_position(df, trial_number)
                           for trial_number, df in enumerate(dataframe_list, start=1)]

    # Concatenate the individual DataFrames into one large DataFrame
    LHandPosition = pd.concat(hand_position_data, ignore_index=False)
    LHandPosition = LHandPosition.reset_index().rename(columns={'index':'timepoint'})
    LHandPosition = LHandPosition[(LHandPosition['timepoint'] >= 0) & (LHandPosition['timepoint'] < 3300)].copy()
    
    LHandPosition['block'] = [1 if x <=72 else 2 if x <=180 else 3 if x <=288 else 4 for x in LHandPosition['trial']]
    
    grouped_df = LHandPosition.groupby('block')

    # Iterate over each block value and group
    for block_value, group_df in grouped_df:
        first_row = group_df.iloc[0]

        # Get the indices of remaining rows
        remaining_indices = group_df.index[1:]

        # Update the original DataFrame with normalized values
        LHandPosition.loc[remaining_indices, 'normalised_x'] = (LHandPosition.loc[remaining_indices, 'hand_position_x'] - first_row['hand_position_x'])*1000
        LHandPosition.loc[remaining_indices, 'normalised_y'] = (LHandPosition.loc[remaining_indices, 'hand_position_y'] - first_row['hand_position_y'])*1000
        LHandPosition.loc[remaining_indices, 'normalised_z'] = (LHandPosition.loc[remaining_indices, 'hand_position_z'] - first_row['hand_position_z'])*1000

    LHandPosition[['normalised_x', 'normalised_y', 'normalised_z']] = LHandPosition[['normalised_x', 'normalised_y', 'normalised_z']].fillna(0)

    LHandPosition = LHandPosition.groupby('trial').mean().drop(['timepoint', 'block', 'hand_position_x',  'hand_position_y',  'hand_position_z'],axis=1)
    
    ##Now the right hand
    def extract_and_interpolate_hand_position(df, trial_number):
        right_hand_position = df['right_hand_position'].apply(pd.Series)  # Expand tuple into separate columns
        interpolated_hand_position = pd.DataFrame({
            'hand_position_x': np.interp(np.arange(baselineLength + trialLength),
                                          df['timestamp'] - df['timestamp'].iloc[0],
                                          right_hand_position[0]),
            'hand_position_y': np.interp(np.arange(baselineLength + trialLength),
                                          df['timestamp'] - df['timestamp'].iloc[0],
                                          right_hand_position[1]),
            'hand_position_z': np.interp(np.arange(baselineLength + trialLength),
                                          df['timestamp'] - df['timestamp'].iloc[0],
                                          right_hand_position[2]),
            'trial': trial_number
        })
        return interpolated_hand_position

    # Apply the function to each DataFrame in the list
    hand_position_data = [extract_and_interpolate_hand_position(df, trial_number)
                           for trial_number, df in enumerate(dataframe_list, start=1)]

    # Concatenate the individual DataFrames into one large DataFrame
    RHandPosition = pd.concat(hand_position_data, ignore_index=False)
    RHandPosition = RHandPosition.reset_index().rename(columns={'index':'timepoint'})
    RHandPosition = RHandPosition[(RHandPosition['timepoint'] >= 0) & (RHandPosition['timepoint'] < 3300)].copy()
    
    
    RHandPosition['block'] = [1 if x <=72 else 2 if x <=180 else 3 if x <=288 else 4 for x in RHandPosition['trial']]
    
    grouped_df = RHandPosition.groupby('block')

    # Iterate over each block value and group
    for block_value, group_df in grouped_df:
        first_row = group_df.iloc[0]

        # Get the indices of remaining rows
        remaining_indices = group_df.index[1:]

        # Update the original DataFrame with normalized values
        RHandPosition.loc[remaining_indices, 'normalised_x'] = (RHandPosition.loc[remaining_indices, 'hand_position_x'] - first_row['hand_position_x'])*1000
        RHandPosition.loc[remaining_indices, 'normalised_y'] = (RHandPosition.loc[remaining_indices, 'hand_position_y'] - first_row['hand_position_y'])*1000
        RHandPosition.loc[remaining_indices, 'normalised_z'] = (RHandPosition.loc[remaining_indices, 'hand_position_z'] - first_row['hand_position_z'])*1000

    RHandPosition[['normalised_x', 'normalised_y', 'normalised_z']] = RHandPosition[['normalised_x', 'normalised_y', 'normalised_z']].fillna(0)

    RHandPosition = RHandPosition.groupby('trial').mean().drop(['timepoint', 'block', 'hand_position_x',  'hand_position_y',  'hand_position_z'],axis=1)
    
    ##now both hands
    
    bothHandPosition = (LHandPosition+RHandPosition)/2
    
    df1 = bothHandPosition.copy().reset_index()
    df = pd.DataFrame(data2load)
    newdf = pd.concat([df,df1],axis=1)

    condMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & ((newdf['condorder'] == 1) | (newdf['condorder'] == 3))]
    Middle_cond_average = calculate_column_wise_average(condMiddle)

    condLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & ((newdf['condorder'] == 1) | (newdf['condorder'] == 3))]
    Left_cond_average = calculate_column_wise_average(condLeft)

    condRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & ((newdf['condorder'] == 1) | (newdf['condorder'] == 3))]
    Right_cond_average = calculate_column_wise_average(condRight) 

    extMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & ((newdf['condorder'] == 2) | (newdf['condorder'] == 4))]
    Middle_ext_average = calculate_column_wise_average(extMiddle)

    extLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & ((newdf['condorder'] == 2) | (newdf['condorder'] == 4))]
    Left_ext_average = calculate_column_wise_average(extLeft)

    extRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & ((newdf['condorder'] == 2) | (newdf['condorder'] == 4))]
    Right_ext_average = calculate_column_wise_average(extRight)    

    x = pd.concat([Middle_cond_average,Left_cond_average,Right_cond_average,
               Middle_ext_average,Left_ext_average,Right_ext_average],
              axis=1)

    x = x.rename(columns={0: 'Middle_cond', 1: 'Left_cond', 2: 'Right_cond', 3: 'Middle_ext', 4: 'Left_ext', 5: 'Right_ext'}).drop('trial',axis=0)
    x = x.reset_index().rename(columns={'index':'Measure'})
    
    x.to_csv(AV_OUT_FILE_NAME)
    
    # Read the CSV file for the current participant
    participant_data = pd.read_csv(AV_OUT_FILE_NAME)
    participant_data["Participant"] = participantID  # Add a column to identify the participant
    all_participants_data = pd.concat([all_participants_data, participant_data], ignore_index=True)

In [27]:
# List of participant IDs
#participant_ids = ["P02", "P03", "P04", "P05", "P06", "P08", "P09", "P10", "P11", "P12", "P13", "P14", "P15", "P16", "P17", "P19", "P20", "P21", "P22", "P24", "P25", "P26", "P27", "P28", "P29","P30"]  # Add more participant IDs as needed
participant_ids = ["02", "03", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "19", "20", "21", "22", "24", "25", "26", "27", "28", "29","30"] 

# Loop over each participant
for participantID in participant_ids:
    # ... (The existing code for processing a single participant will go here)
    process_participant(participantID)
    
# Concatenate all participant data into a single DataFrame
all_participants_data.to_csv("/Users/dhewitt/Data/pps/Exports/Combined_hand_data.csv", index=False)